In [74]:
import pandas as pd
import plotly


In [75]:

#read with pandas all the csv file in ./data/train/group0
df0= pd.read_csv('../data/train/group0/dataset_user_0_train.csv', sep=';')
df1= pd.read_csv('../data/train/group0/dataset_user_9_train.csv', sep=';')
df18= pd.read_csv('../data/train/group0/dataset_user_18_train.csv', sep=';')
df27= pd.read_csv('../data/train/group0/dataset_user_27_train.csv', sep=';')
df36= pd.read_csv('../data/train/group0/dataset_user_36_train.csv', sep=';')


#print of the single value of the firrst row of df0
print(df18.iloc[1])


df0=df18

Unnamed: 0                                                                               1
day                                                                                      1
label                                                                                   64
hr_maxHeartRate                                                                        101
hr_minHeartRate                                                                         45
hr_restingHeartRate                                                                     47
hr_lastSevenDaysAvgRestingHeartRate                                                     65
hr_time_series                           [None, 80, 66, 55, 55, 52, 57, 60, 59, 51, 55,...
resp_lowestRespirationValue                                                            6.0
resp_highestRespirationValue                                                          19.0
resp_avgWakingRespirationValue                                                        13.0

In [76]:
import pandas as pd
import plotly
from pathlib import Path
import re

base = Path('../data/train')

groups_dfs = {}

for csv_path in sorted(base.glob('group*/*.csv')):
    group = csv_path.parent.name
    m = re.search(r'dataset_user_(\d+)_train\.csv', csv_path.name)
    if not m:
        continue
    user_id = int(m.group(1))
    df = pd.read_csv(csv_path, sep=';')
    groups_dfs.setdefault(group, {})[user_id] = df


In [77]:
#for each person in each group i want to know how many rows are in the dataframe
total_users = 0
nan_activity_count=0
for group, users in groups_dfs.items():
    total_users += 1
    for user_id, df in users.items():
        print(f'Group: {group}, User: {user_id}, Rows: {len(df)}')
        # how many rows have Nan values and for wich columns
        nan_counts = df.isna().sum()
        # print(f'NaN counts:\n{nan_counts[nan_counts > 0]}')
        # if there are columns that are all NaN
        all_nan_columns = nan_counts[nan_counts == len(df)].index.tolist()
        if all_nan_columns:
            print(f'Columns with all NaN values: {all_nan_columns}')
            nan_activity_count+=1

#all the users have the column act_activeTime empty so we can drop it
for group, users in groups_dfs.items():
    for user_id, df in users.items():
        if 'act_activeTime' in df.columns:
            df.drop(columns=['act_activeTime'], inplace=True)


Group: group0, User: 0, Rows: 27
Columns with all NaN values: ['act_activeTime']
Group: group0, User: 18, Rows: 25
Columns with all NaN values: ['act_activeTime']
Group: group0, User: 27, Rows: 4
Columns with all NaN values: ['act_activeTime']
Group: group0, User: 36, Rows: 22
Columns with all NaN values: ['act_activeTime']
Group: group0, User: 9, Rows: 29
Columns with all NaN values: ['act_activeTime']
Group: group1, User: 10, Rows: 26
Columns with all NaN values: ['act_activeTime']
Group: group1, User: 19, Rows: 13
Columns with all NaN values: ['act_activeTime']
Group: group1, User: 1, Rows: 22
Columns with all NaN values: ['act_activeTime']
Group: group1, User: 28, Rows: 9
Columns with all NaN values: ['act_activeTime']
Group: group1, User: 37, Rows: 24
Columns with all NaN values: ['act_activeTime']
Group: group2, User: 11, Rows: 20
Columns with all NaN values: ['act_activeTime']
Group: group2, User: 20, Rows: 13
Columns with all NaN values: ['act_activeTime']
Group: group2, User: 

In [ ]:
import ast
import pandas as pd


cleaned_data = {}


for group, users in groups_dfs.items():

    cleaned_data[group] = {}

    for user_id, df in users.items():

        df['len_hr_time_series'] = df['hr_time_series'].apply(lambda x: len(ast.literal_eval(x)) if pd.notna(x) else 0)
        df['len_resp_time_series'] = df['resp_time_series'].apply(lambda x: len(ast.literal_eval(x)) if pd.notna(x) else 0)
        df['len_stress_time_series'] = df['stress_time_series'].apply(lambda x: len(ast.literal_eval(x)) if pd.notna(x) else 0)

        coverage_hr_list=[]
        coverage_resp_list=[]
        coverage_stress_list=[]

        for n in range(len(df)):
            len_hr=df.iloc[n]['len_hr_time_series']
            len_resp=df.iloc[n]['len_resp_time_series']
            len_stress=df.iloc[n]['len_stress_time_series']


            '''
                in the hr-time_series the invald values are None 
                in the resp-time_series the invald values are -1/ -2
                in the stress-time_series the invald values are -1/ -2

            '''


            len_valid_hr=0
            len_valid_resp=0
            len_valid_stress=0
            hr_series=ast.literal_eval(df.iloc[n]['hr_time_series']) if pd.notna(df.iloc[n]['hr_time_series']) else []
            resp_series=ast.literal_eval(df.iloc[n]['resp_time_series']) if pd.notna(df.iloc[n]['resp_time_series']) else []
            stress_series=ast.literal_eval(df.iloc[n]['stress_time_series']) if pd.notna(df.iloc[n]['stress_time_series']) else []  
            for i in range(len_hr):
                if hr_series[i] is not None:
                    len_valid_hr+=1
            for i in range(len_resp):
                if resp_series[i]!=-1 and resp_series[i]!=-2:
                    len_valid_resp+=1
            for i in range(len_stress):
                if stress_series[i]!=-1 and stress_series[i]!=-2:
                    len_valid_stress+=1
            coverage_hr=len_valid_hr/len_hr if len_hr>0 else 0
            coverage_resp=len_valid_resp/len_resp if len_resp>0 else 0
            coverage_stress=len_valid_stress/len_stress if len_stress>0 else 0
            coverage_hr_list.append(coverage_hr)
            coverage_resp_list.append(coverage_resp)
            coverage_stress_list.append(coverage_stress)
            # print(f'Row {n}: coverage_hr={coverage_hr}, coverage_resp={coverage_resp}, coverage_stress={coverage_stress}')
            # print(f'len_hr={len_hr}, len_resp={len_resp}, len_stress={len_stress}')
            # print('--------------------------------------------')


        df['coverage_hr']=coverage_hr_list
        df['coverage_resp']=coverage_resp_list
        df['coverage_stress']=coverage_stress_list


        MIN_COVERAGE=0.75
        #take only the rows where coverage_hr, coverage_resp, coverage_stress are all > MIN_COVERAGE
        df_cleaned = df[(df['coverage_hr'] >= MIN_COVERAGE) &
                        (df['coverage_resp'] >= MIN_COVERAGE) &
                        (df['coverage_stress'] >= MIN_COVERAGE)]    
        cleaned_data[group][user_id] = df_cleaned






Group: group0, User: 0
Group: group0, User: 18
Group: group0, User: 27
Group: group0, User: 36
Group: group0, User: 9
Group: group1, User: 10
Group: group1, User: 19
Group: group1, User: 1
Group: group1, User: 28
Group: group1, User: 37
Group: group2, User: 11
Group: group2, User: 20
Group: group2, User: 29
Group: group2, User: 2
Group: group2, User: 38
Group: group3, User: 12
Group: group3, User: 21
Group: group3, User: 30
Group: group3, User: 39
Group: group3, User: 3
Group: group4, User: 13
Group: group4, User: 22
Group: group4, User: 31
Group: group4, User: 40
Group: group4, User: 4
Group: group5, User: 14
Group: group5, User: 23
Group: group5, User: 32
Group: group5, User: 41
Group: group5, User: 5
Group: group6, User: 15
Group: group6, User: 24
Group: group6, User: 33
Group: group6, User: 42
Group: group6, User: 6
Group: group7, User: 16
Group: group7, User: 25
Group: group7, User: 34
Group: group7, User: 43
Group: group7, User: 7
Group: group8, User: 17
Group: group8, User: 26
G

In [79]:
# compare the difference in number of rows before and after cleaning
for group, users in groups_dfs.items():
    
    for user_id, df in users.items():
        df_clean = cleaned_data[group][user_id]
        original_rows = len(df)
        cleaned_rows = len(df_clean)
        print(f'Group: {group}, User: {user_id}, Original Rows: {original_rows}, Cleaned Rows: {cleaned_rows}, Removed: {original_rows - cleaned_rows}')

Group: group0, User: 0, Original Rows: 27, Cleaned Rows: 19, Removed: 8
Group: group0, User: 18, Original Rows: 25, Cleaned Rows: 0, Removed: 25
Group: group0, User: 27, Original Rows: 4, Cleaned Rows: 2, Removed: 2
Group: group0, User: 36, Original Rows: 22, Cleaned Rows: 3, Removed: 19
Group: group0, User: 9, Original Rows: 29, Cleaned Rows: 17, Removed: 12
Group: group1, User: 10, Original Rows: 26, Cleaned Rows: 8, Removed: 18
Group: group1, User: 19, Original Rows: 13, Cleaned Rows: 4, Removed: 9
Group: group1, User: 1, Original Rows: 22, Cleaned Rows: 19, Removed: 3
Group: group1, User: 28, Original Rows: 9, Cleaned Rows: 9, Removed: 0
Group: group1, User: 37, Original Rows: 24, Cleaned Rows: 15, Removed: 9
Group: group2, User: 11, Original Rows: 20, Cleaned Rows: 10, Removed: 10
Group: group2, User: 20, Original Rows: 13, Cleaned Rows: 5, Removed: 8
Group: group2, User: 29, Original Rows: 25, Cleaned Rows: 20, Removed: 5
Group: group2, User: 2, Original Rows: 14, Cleaned Rows: 7

In [ ]:
import os

BASE_DIR = "filtered_data"

# cartella principale
os.makedirs(BASE_DIR, exist_ok=True)

for group, users in cleaned_data.items():
    group_dir = os.path.join(BASE_DIR, str(group))
    os.makedirs(group_dir, exist_ok=True)

    for user_id, df in users.items():
        if df.empty:
            continue  # opzionale: salta utenti senza row valide

        file_path = os.path.join(group_dir, f"dataset_user_{user_id}_train.csv")
        df.to_csv(file_path, index=False)